In [1]:
import torch
import cv2
import pyttsx3
import requests
import json
import numpy as np
import time
import speech_recognition as sr
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import openai

In [2]:
#Loading pre-trained YOLO model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained = True)

Using cache found in C:\Users\Administrator/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-6-2 Python-3.10.7 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [3]:
#Initialize Text-to-Speech Engine
engine = pyttsx3.init()

In [8]:
#Initialize OpenAI API(Use your Own People)
openai.api_key = 'Key'

In [9]:
def speak(text):
    engine.say(text)
    engine.runAndWait()

In [10]:
def get_objects_details(object_name):
    try:
        url = f"htps://end.wikipedia.org/api/rest_v1/page/summary/{object_name}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            return data.get('extract', f"Information about {object_name}")
        else:
            return f"Information about {object_name} is not available."
    except requests.RequestException as e:
        return f"Could not retrieve information due to an error: {str(e)}."

In [11]:
def get_position(bbox, frame_width):
    center_x = (bbox[0] + bbox[2]) /2
    if center_x < frame_width / 3:
        return "left"
    elif center_x > 2 *frame_width /3:
        return "right"
    else:
        return "center"